In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
directory = "/Users/isabelagaloustian/Library/CloudStorage/OneDrive-DrexelUniversity/General - Chang Lab/Group/Data/Gigafactory and Mining Data/"

minerals = ["Manganese", "Lithium", "Cobalt", "Nickel"]

combined = None

filepath = os.path.join(directory, "SPGlobal_MetalsAndMiningProperties-Combined-Dec-2024.xlsx")
for i, mineral in enumerate(minerals):
    reserves_data = pd.read_excel(filepath, sheet_name = mineral)
    # making dates in excel sheet readable by pandas
    reserves_data["Reserves & Resources As Of Date"] = pd.to_datetime(reserves_data["Reserves & Resources As Of Date"])
    reserves_data["Primary Mineral"] = mineral
    # reformating dates to year only, then make new column 
    reserves_data["Year"] = reserves_data["Reserves & Resources As Of Date"].dt.year

    if i == 0:
        combined = reserves_data
        continue

    combined = pd.concat([combined, reserves_data], ignore_index=True)

combined = combined.sort_values(by = "Year").query("`Activity Status` == 'Active'")
combined["Property ID"] = combined["Property ID"].astype(int)

locations = pd.read_excel("/Users/isabelagaloustian/Library/CloudStorage/OneDrive-DrexelUniversity/General - Chang Lab/Group/Data/Gigafactory and Mining Data/MiningPropertyLocations.xlsx")
coords = locations[["KeyMineProject","Latitude (degrees)", "Longitude (degrees)"]]

combined = combined.merge(coords, left_on = "Property ID", right_on="KeyMineProject" ,how = "inner")
combined.drop("KeyMineProject", axis=1, inplace=True, errors="ignore")

start_year = 2010
combined = combined[combined["Year"] >= start_year]

D = combined[["Year","Primary Mineral", "Primary Reserves and Resources"]]

D = D.groupby(["Year","Primary Mineral"], as_index=False).agg("sum")


# Generate a complete set of Year-Mineral combinations
all_years = D["Year"].unique()
all_minerals = D["Primary Mineral"].unique()
complete_index = pd.MultiIndex.from_product([all_years, all_minerals], names=["Year", "Primary Mineral"])

# Reindex the DataFrame to include all combinations, filling missing values with 0
D = D.set_index(["Year", "Primary Mineral"]).reindex(complete_index, fill_value=0).reset_index()

D["Cumulative Reserves"] = D.groupby(["Primary Mineral"], as_index=False)["Primary Reserves and Resources"].agg("cumsum")

D


,Year,Primary Mineral,Primary Reserves and Resources,Cumulative Reserves
0,2010,Cobalt,22369,22369
1,2010,Lithium,408140,408140
2,2010,Nickel,1518795,1518795
3,2010,Manganese,0,0
4,2011,Cobalt,651,23020
5,2011,Lithium,11231,419371
6,2011,Nickel,3094440,4613235
7,2011,Manganese,0,0
8,2012,Cobalt,334526,357546
9,2012,Lithium,929949,1349320
